# INM706 - Deep Learning for Sequence Analysis

Authors: Laerte Adami - Elisa Troschka

In [1]:
from Utilities.lstmHandler import EncoderLSTM, DecoderLSTM
from Utilities.modelHandler import LSTModel
from Utilities.LanguageDataset import LanguageDataset
from torch.utils.data import DataLoader

import torch
from torch.nn import CrossEntropyLoss as CEL
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
dataset = LanguageDataset(data_path="./Data/eng_ita.tsv")

vocabulary_size = dataset.eng_voc_size
embedding_size = 256

bos_token = '<BoS>'
eos_token = '<EoS>'

loss_func = CEL()

In [6]:
trainloader = DataLoader(dataset, batch_size=1)

In [7]:
batch_size = 1
x, y = next(iter(trainloader))
prev_hidden = torch.zeros(1, batch_size, embedding_size, device = device)

In [10]:
out, hidden = encoder(x)

TypeError: forward() missing 1 required positional argument: 'prev_hidden'

In [9]:
encoder = EncoderLSTM(vocabulary_size = vocabulary_size,
                     embedding_size = embedding_size,
                     num_layers = 1, 
                     bidirectional = False)

decoder = DecoderLSTM(vocabulary_size = vocabulary_size,
                     embedding_size = embedding_size,
                     num_layers = 1, 
                     bidirectional = False)

In [4]:
model = LSTModel(encoder = encoder, 
                 decoder = decoder, 
                 encoder_optimizer = Adam(encoder.parameters()), 
                 decoder_optimizer = Adam(decoder.parameters()),
                 loss_function = loss_func, 
                 eos_token = eos_token, 
                 bos_token = bos_token)

In [8]:
print(decoder)

DecoderLSTM(
  (emb): Embedding(100, 30)
  (lstm): LSTM(30, 30, batch_first=True)
  (relu): ReLU()
)


In [6]:
z = torch.zeros(2,3,4)

In [11]:
zz = z[:,-1,:].unsqueeze(1)

In [12]:
zz.size()

torch.Size([2, 1, 4])

In [1]:
import torch.nn as nn

In [4]:
class EncoderLSTM(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_size, num_layers, bidirectional):
        
        super(EncoderLSTM, self).__init__()
        
        self.emb = nn.Embedding(vocabulary_size, embedding_size)
        self.lstm = nn.lstm(input_size = embedding_size, hidden_size = embedding_size, 
                            num_layers = num_layers, batch_first = True, bidirectional = bidirectional)
        

        
    def forward(self, x, prev_hidden):
        
        x = self.emb(x)
        output, hidden = self.lstm(x, prev_hidden)
        
        return output, hidden
    
class DecoderLSTM(nn.Module):
    
    def __init__(self, vocabulary_size, embedding_size, num_layers, bidirectional):

        super(DecoderLSTM, self).__init__()
        
        self.emb = nn.Embedding(vocabulary_size, embedding_size)
        self.lstm = nn.lstm(input_size = embedding_size, hidden_size = embedding_size, 
                            num_layers = num_layers, batch_first = True, bidirectional = bidirectional)
        self.relu = nn.ReLU(inplace=False)
        
    def forward(self, x, prev_hidden):
        
        x = self.relu(x)
        output, hidden = self.lstm(x, prev_hidden)
        
        return output, hidden
        
        
        
        
        
        
        
        